In [1]:
!git clone https://ChristineTharian@bitbucket.org/ChristineTharian/deeplearning.git

Cloning into 'deeplearning'...
remote: Counting objects: 10, done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 10 (delta 1), reused 0 (delta 0)
Unpacking objects: 100% (10/10), done.


In [0]:
from scipy.io import loadmat
import pandas as pd
import numpy as np
from random import shuffle
import os
import cv2

In [0]:
image_size = (48,48)

In [0]:
data = pd.read_csv('/content/deeplearning/fer2013.csv')

In [0]:
data['emotion'] = data.emotion.apply(lambda x: 1 if x == 3 else 0)

In [9]:
#data = pd.read_csv('/content/deeplearning/fer2013.csv')
pixels = data['pixels'].tolist()
width, height = 48, 48
faces = []
for pixel_sequence in pixels:
    face = [int(pixel) for pixel in pixel_sequence.split(' ')]
    face = np.asarray(face).reshape(width, height)
    face = cv2.resize(face.astype('uint8'), image_size)
    faces.append(face.astype('float32'))
faces = np.asarray(faces)
faces = np.expand_dims(faces, -1)
emotions = pd.get_dummies(data['emotion']).as_matrix()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # This is added back by InteractiveShellApp.init_path()


In [10]:
data['emotion'].value_counts()

0    26898
1     8989
Name: emotion, dtype: int64

In [11]:
import sys, os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2

Using TensorFlow backend.


In [0]:
num_features = 64
#num_labels = 7
num_labels = 2
batch_size = 64
epochs = 100
width, height = 48, 48

x = faces
y = emotions

def preprocess_input(x, v2=True):
    x = x.astype('float32')
    x = x / 255.0
    if v2:
        x = x - 0.5
        x = x * 2.0
    return x

# x -= np.mean(x, axis=0)
# x /= np.std(x, axis=0)

preprocess_input(x)

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=41)

In [13]:
model = Sequential()

model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1), data_format='channels_last', kernel_regularizer=l2(0.01)))
model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(2*2*2*num_features, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(2*2*num_features, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(2*num_features, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(num_labels, activation='softmax'))

model.summary()













Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 46, 46, 64)        640       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 46, 46, 64)        36928     
_________________________________________________________________
batch_normalization_1 (Batch (None, 46, 46, 64)        256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 23, 23, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 23, 23, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 23, 23, 128)       7385

In [14]:
model.compile(loss=categorical_crossentropy,
              optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-7),
              metrics=['accuracy'])

#training the model



# fit model
n_epochs, n_save_after = 100, 70
for i in range(n_epochs):
	# fit model for a single epoch
  print(i)
  model.fit(X_train, y_train, epochs=1, verbose=1, validation_data=(np.array(X_valid), np.array(y_valid)))
	# check if we should save the model
  if i >= n_save_after:
  	model.save('model_' + str(i) + '.h5')
# model.fit(np.array(X_train), np.array(y_train),
#           batch_size=batch_size,
#           epochs=30,
#           verbose=1,
#           validation_data=(np.array(X_valid), np.array(y_valid)),
#           shuffle=True)

# #saving the  model to be used later
# fer_json = model.to_json()
# with open("fer.json", "w") as json_file:
#     json_file.write(fer_json)
# model.save_weights("fer.h5")
# print("Saved model to disk")



0
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 29068 samples, validate on 3230 samples
Epoch 1/1
29068/29068 [==============================] - 84s 3ms/step - loss: 0.7134 - acc: 0.7264 - val_loss: 0.5935 - val_acc: 0.7406
1
Train on 29068 samples, validate on 3230 samples
Epoch 1/1
29068/29068 [==============================] - 76s 3ms/step - loss: 0.5766 - acc: 0.7493 - val_loss: 0.5829 - val_acc: 0.7406
2
Train on 29068 samples, validate on 3230 samples
Epoch 1/1
29068/29068 [==============================] - 76s 3ms/step - loss: 0.5123 - acc: 0.7673 - val_loss: 0.4580 - val_acc: 0.8266
3
Train on 29068 samples, validate on 3230 samples
Epoch 1/1
29068/29068 [==============================] - 76s 3ms/step - loss: 0.3984 - acc: 0.8336 - val_loss: 0.3388 - val_acc: 0.8582
4
Train on 29068 samples, validate on 3230 samples
Epoch 1/1
29068/29068 [==============================] - 76s 3ms/step - loss: 0.3423 - acc: 0.8642 - va

In [0]:
model.save('hope.h5')

In [0]:
model.save_weights('old.h5')

In [0]:
from keras.models import load_model

def load_all_models(n_start, n_end):
	all_models = list()
	for epoch in range(n_start, n_end):
		# define filename for this ensemble
		filename = 'model_' + str(epoch) + '.h5'
		# load model from file
		model = load_model(filename)
		# add to list of members
		all_models.append(model)
		print('>loaded %s' % filename)
	return all_models

In [0]:
def ensemble_predictions(members, testX):
	# make predictions
	yhats = [model.predict(testX, verbose = 1) for model in members]
	yhats = np.array(yhats)
	# sum across ensemble members
	summed = np.sum(yhats, axis=0)
	# argmax across classes
	result = np.argmax(summed, axis=1)
	return result

In [0]:
members = load_all_models(70, 100)
preds = ensemble_predictions(members, X_test)
temp = sum(np.argmax(y_test, axis=1) == preds)
temp/len(y_test) 

>loaded model_30.h5
>loaded model_31.h5
>loaded model_32.h5
>loaded model_33.h5
>loaded model_34.h5
>loaded model_35.h5
>loaded model_36.h5
>loaded model_37.h5
>loaded model_38.h5
>loaded model_39.h5
>loaded model_40.h5
>loaded model_41.h5
>loaded model_42.h5
>loaded model_43.h5
>loaded model_44.h5
>loaded model_45.h5
>loaded model_46.h5
>loaded model_47.h5
>loaded model_48.h5
>loaded model_49.h5
3589/3589 [==============================] - 10s 3ms/step


0.6609083310114238